# Team

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from sqlalchemy import create_engine
from geojson import Feature, FeatureCollection, Point

In [2]:
Coordinates = pd.read_csv('Resources/world_country_latitude_and_longitude_values.csv')
Coordinates

,Country,Latitude,Longitude
0,Andorra,42.50,1.50
1,United Arab Emirates,24.00,54.00
2,Afghanistan,33.00,65.00
3,Antigua and Barbuda,17.05,-61.80
4,Anguilla,18.25,-63.17
...,...,...,...
235,Yemen,15.00,48.00
236,Mayotte,-12.83,45.17
237,South Africa,-29.00,24.00
238,Zambia,-15.00,30.00


In [3]:
Team_data = pd.read_csv('Resources/Footballteams.csv')
Team_data['Country']=Team_data['Country'].apply(lambda x: x.strip())
Team_data

,Team,Tournament,Goals,Shots pg,yellow_cards,red_cards,Possession%,Pass%,AerialsWon,Rating,Country
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01,England
1,Bayern Munich,Bundesliga,99,17.1,44,3,58.1,85.5,12.9,6.95,Germany
2,Paris Saint-Germain,Ligue 1,86,15.0,73,7,60.1,89.5,9.5,6.88,France
3,Barcelona,LaLiga,85,15.3,68,2,62.4,89.7,10.6,6.87,Spain
4,Real Madrid,LaLiga,67,14.4,57,2,57.7,87.7,11.8,6.86,Spain
...,...,...,...,...,...,...,...,...,...,...,...
93,Sheffield United,Premier League,20,8.5,73,3,43.0,76.9,19.1,6.46,England
94,Crotone,Serie A,45,9.5,85,4,47.2,80.4,12.7,6.43,Italy
95,Benevento,Serie A,40,11.0,90,5,44.2,77.7,13.4,6.43,Italy
96,Dijon,Ligue 1,25,9.2,75,5,46.9,80.0,14.3,6.42,France


In [17]:
Team_data_Final = pd.merge(Team_data,Coordinates,on = "Country")
Team_data_Final

,Team,Tournament,Goals,Shots pg,yellow_cards,red_cards,Possession%,Pass%,AerialsWon,Rating,Country,Latitude,Longitude
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01,England,54.00,-2.00
1,Manchester United,Premier League,73,13.8,64,1,54.5,84.8,14.5,6.85,England,54.00,-2.00
2,Aston Villa,Premier League,55,13.7,63,4,49.1,78.6,19.4,6.84,England,54.00,-2.00
3,Chelsea,Premier League,58,14.6,49,3,58.6,87.0,15.2,6.83,England,54.00,-2.00
4,Liverpool,Premier League,68,16.0,40,0,59.0,85.7,14.3,6.82,England,54.00,-2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Fiorentina,Serie A,47,9.8,86,5,46.8,81.0,14.2,6.52,Italy,42.83,12.83
94,Spezia,Serie A,52,10.2,92,5,51.6,81.1,14.4,6.50,Italy,42.83,12.83
95,Parma Calcio 1913,Serie A,39,10.4,91,1,48.4,82.5,16.9,6.49,Italy,42.83,12.83
96,Crotone,Serie A,45,9.5,85,4,47.2,80.4,12.7,6.43,Italy,42.83,12.83


In [5]:
# columns used for constructing geojson object
features = Team_data_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = Team_data_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [17]:
with open('Team_Data.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [4]:
engine = create_engine('sqlite:///football_team_data.db')

In [5]:
Team_data.to_sql('premier_league_team_data', con=engine, if_exists='replace', index=False)

In [6]:
engine.execute("SELECT * FROM premier_league_team_data").fetchall()

[('Manchester City', 'Premier League', 83, 15.8, 46, 2, 60.8, 89.4, 12.8, 7.01, 'England '),
 ('Bayern Munich', 'Bundesliga', 99, 17.1, 44, 3, 58.1, 85.5, 12.9, 6.95, 'Germany '),
 ('Paris Saint-Germain', 'Ligue 1', 86, 15.0, 73, 7, 60.1, 89.5, 9.5, 6.88, 'France '),
 ('Barcelona', 'LaLiga', 85, 15.3, 68, 2, 62.4, 89.7, 10.6, 6.87, 'Spain'),
 ('Real Madrid', 'LaLiga', 67, 14.4, 57, 2, 57.7, 87.7, 11.8, 6.86, 'Spain'),
 ('Manchester United', 'Premier League', 73, 13.8, 64, 1, 54.5, 84.8, 14.5, 6.85, 'England '),
 ('Juventus', 'Serie A', 77, 15.7, 76, 6, 55.4, 88.3, 11.4, 6.85, 'Italy '),
 ('Aston Villa', 'Premier League', 55, 13.7, 63, 4, 49.1, 78.6, 19.4, 6.84, 'England '),
 ('Borussia Dortmund', 'Bundesliga', 75, 14.6, 43, 1, 57.5, 85.5, 12.8, 6.84, 'Germany '),
 ('Atletico Madrid', 'LaLiga', 67, 12.1, 100, 0, 51.8, 83.1, 14.4, 6.84, 'Spain'),
 ('Atalanta', 'Serie A', 90, 16.3, 66, 3, 53.5, 83.5, 16.8, 6.84, 'Italy '),
 ('Chelsea', 'Premier League', 58, 14.6, 49, 3, 58.6, 87.0, 15.2, 

In [ ]:
Country_Rating = Team_data.groupby("Country").agg({"Rating":["sum"]})
Country_Goals = Team_data.groupby("Country").agg({"Goals":["sum"]})
Country_Shots = Team_data.groupby("Country").agg({"Shots pg":["sum"]})
Country_yellow_cards = Team_data.groupby("Country").agg({"yellow_cards":["sum"]})
Country_red_cards = Team_data.groupby("Country").agg({"red_cards":["sum"]})

Country_summary1 = pd.merge(Country_Rating,Country_Goals,on = "Country")
Country_summary2 = pd.merge(Country_summary1,Country_Shots,on = "Country")
Country_summary3 = pd.merge(Country_summary2,Country_yellow_cards,on = "Country")
Country_summary = pd.merge(Country_summary3,Country_red_cards,on = "Country")

#Reset the Index
Country_summary.reset_index(inplace=True)
#Assign name for the column variables
Country_summary.columns = ["Country","Rating","Goals","Shots","Yellow_Cards","Red_Cards"]

In [ ]:
football_teams_summary = pd.merge(Country_summary,Coordinates,on = "Country")
football_teams_summary.to_csv(r'Country_summary.csv')
football_teams_summary

import csv, json

csvFilePath = "Country_summary.csv"  ## path to file
jsonFilePath = "Country_summary.json"   ## change to wanted file name

data = {}
with open(csvFilePath) as csvFile:
    csvReader = csv.DictReader(csvFile)
    for rows in csvReader:
        id = rows['Country']
        data[id] = rows

with open(jsonFilePath, 'w') as jsonFile:
    jsonFile.write(json.dumps(data, indent=4))

In [11]:
chelsea_players = pd.read_csv('Resources/chelsea_players.csv')
chelsea_players_Final = pd.merge(chelsea_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = chelsea_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = chelsea_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('chelsea_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

chelsea_players_Final

,Player,Club,Nationality,Country,Latitude,Longitude
0,Edouard Mendy,Chelsea FC,Senegal,Senegal,14.00,-14.00
1,Kepa,Chelsea FC,Spain,Spain,40.00,-4.00
2,Kurt Zouma,Chelsea FC,France,France,46.00,2.00
3,Andreas Christensen,Chelsea FC,Denmark,Denmark,56.00,10.00
4,Antonio Rüdiger,Chelsea FC,Germany,Germany,51.00,9.00
5,Ethan Ampadu,Chelsea FC,Wales,NaN,NaN,NaN
6,Thiago Silva,Chelsea FC,Brazil,Brazil,-10.00,-55.00
7,Jake Clarke-Salter,Chelsea FC,England,England,54.00,-2.00
8,Ben Chilwell,Chelsea FC,England,England,54.00,-2.00
9,Marcos Alonso,Chelsea FC,Spain,Spain,40.00,-4.00


In [ ]:
manchester_players = pd.read_csv('Resources/manchester_players.csv')
psg_players = pd.read_csv('Resources/psg_players.csv')
madrid_players = pd.read_csv('Resources/real_madrid_players.csv')
chelsea_players

In [14]:
manchester_players = pd.read_csv('Resources/manchester_players.csv')
manchester_players_Final = pd.merge(manchester_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = manchester_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = manchester_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('manchester_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

manchester_players_Final

,Player,Club,Nationality,Country,Latitude,Longitude
0,Ederson,Manchester City,Brazil,Brazil,-10.00,-55.00
1,Zack Steffen,Manchester City,United States,United States,38.00,-97.00
2,Rúben Dias,Manchester City,Portugal,Portugal,39.50,-8.00
3,Aymeric Laporte,Manchester City,Spain,Spain,40.00,-4.00
4,Nathan Aké,Manchester City,Netherlands,Netherlands,52.50,5.75
5,John Stones,Manchester City,England,England,54.00,-2.00
6,Philippe Sandler,Manchester City,Netherlands,Netherlands,52.50,5.75
7,Oleksandr Zinchenko,Manchester City,Ukraine,Ukraine,49.00,32.00
8,Benjamin Mendy,Manchester City,France,France,46.00,2.00
9,João Cancelo,Manchester City,Portugal,Portugal,39.50,-8.00


In [15]:
psg_players = pd.read_csv('Resources/psg_players.csv')
psg_players_Final = pd.merge(psg_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = psg_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = psg_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('psg_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

psg_players_Final

,Player,Club,Nationality,Country,Latitude,Longitude
0,Keylor Navas,Paris Saint-Germain,Costa Rica,Costa Rica,10.00,-84.00
1,Sergio Rico,Paris Saint-Germain,Spain,Spain,40.00,-4.00
2,Marcin Bulka,Paris Saint-Germain,Poland,Poland,52.00,20.00
3,Alexandre Letellier,Paris Saint-Germain,France,France,46.00,2.00
4,Garissone Innocent,Paris Saint-Germain,Haiti,Haiti,19.00,-72.42
5,Denis Franchi,Paris Saint-Germain,Italy,Italy,42.83,12.83
6,Marquinhos,Paris Saint-Germain,Brazil,Brazil,-10.00,-55.00
7,Presnel Kimpembe,Paris Saint-Germain,France,France,46.00,2.00
8,Thilo Kehrer,Paris Saint-Germain,Germany,Germany,51.00,9.00
9,Abdou Diallo,Paris Saint-Germain,Senegal,Senegal,14.00,-14.00


In [7]:
madrid_players = pd.read_csv('Resources/real_madrid_players.csv')
madrid_players_Final = pd.merge(madrid_players,Coordinates,how='left', left_on=['Nationality'], right_on=['Country'])

# columns used for constructing geojson object
features = madrid_players_Final.apply(
    lambda row: Feature(geometry=Point((float(row['Longitude']), float(row['Latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = madrid_players_Final.drop(['Latitude', 'Longitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

with open('manchester_players_Final.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

madrid_players

,Player,Club,Nationality,Coordinates
0,Thibaut Courtois,Real Madrid,Belgium,"50.503887, 4.469936"
1,<<<<<<< HEAD,NaN,NaN,NaN
2,Andriy Lunin,Real Madrid,Ukraine,"48.379433,-31.16557999"
3,=======,NaN,NaN,NaN
4,Andriy Lunin,Real Madrid,Ukraine,"48.379433, 31.16558"
5,>>>>>>> 4a3c52d7ddb6079f3f49ffa21f84783b91971b27,NaN,NaN,NaN
6,Raphaël Varane,Real Madrid,France,"46.227638, 2.213749"
7,Éder Militão,Real Madrid,Brazil,"-14.235004, -51.92528"
8,Nacho Fernández,Real Madrid,Spain,"40.463667, -3.74922"
9,Ferland Mendy,Real Madrid,France,"46.227638, 2.213749"


In [5]:
chelsea_group = chelsea_players.groupby("Nationality").agg({"Nationality":["count"]})
#nationality_group.reset_index(inplace=True)
#nationality_group.columns = ["Nationality","Count"]
#Sorted_nationalities = nationality_group.sort_values(by = "Count", ascending = False)
#Sorted_nationalities

In [6]:
manchester_players

,Player,Club,Nationality
0,Ederson,Manchester City,Brazil
1,Zack Steffen,Manchester City,United States
2,Rúben Dias,Manchester City,Portugal
3,Aymeric Laporte,Manchester City,Spain
4,Nathan Aké,Manchester City,Netherlands
5,John Stones,Manchester City,England
6,Philippe Sandler,Manchester City,Netherlands
7,Oleksandr Zinchenko,Manchester City,Ukraine
8,Benjamin Mendy,Manchester City,France
9,João Cancelo,Manchester City,Portugal


In [7]:
psg_players

,Player,Club,Nationality
0,Keylor Navas,Paris Saint-Germain,Costa Rica
1,Sergio Rico,Paris Saint-Germain,Spain
2,Marcin Bulka,Paris Saint-Germain,Poland
3,Alexandre Letellier,Paris Saint-Germain,France
4,Garissone Innocent,Paris Saint-Germain,Haiti
5,Denis Franchi,Paris Saint-Germain,Italy
6,Marquinhos,Paris Saint-Germain,Brazil
7,Presnel Kimpembe,Paris Saint-Germain,France
8,Thilo Kehrer,Paris Saint-Germain,Germany
9,Abdou Diallo,Paris Saint-Germain,Senegal


In [8]:
madrid_players

,Player,Club,Nationality
0,Thibaut Courtois,Real Madrid,Belgium
1,Andriy Lunin,Real Madrid,Ukraine
2,Raphaël Varane,Real Madrid,France
3,Éder Militão,Real Madrid,Brazil
4,Nacho Fernández,Real Madrid,Spain
5,Ferland Mendy,Real Madrid,France
6,Marcelo,Real Madrid,Brazil
7,Daniel Carvajal,Real Madrid,Spain
8,Álvaro Odriozola,Real Madrid,Spain
9,Casemiro,Real Madrid,Brazil


In [9]:
frames = [chelsea_players, manchester_players, psg_players,madrid_players]
all_nationalities = pd.concat(frames)
all_nationalities

,Player,Club,Nationality
0,Edouard Mendy,Chelsea FC,Senegal
1,Kepa,Chelsea FC,Spain
2,Kurt Zouma,Chelsea FC,France
3,Andreas Christensen,Chelsea FC,Denmark
4,Antonio Rüdiger,Chelsea FC,Germany
...,...,...,...
18,Rodrygo,Real Madrid,Brazil
19,Lucas Vázquez,Real Madrid,Spain
20,Karim Benzema,Real Madrid,France
21,Luka Jovic,Real Madrid,Serbia


In [10]:
nationality_group = all_nationalities.groupby(["Club","Nationality"]).agg({"Nationality":["count"]})
nationality_group.reset_index(inplace=True)
nationality_group.columns = ["Club","Nationality","Count"]
Sorted_nationalities = nationality_group.sort_values(by = "Club", ascending = False)
Sorted_nationalities

,Club,Nationality,Count
46,Real Madrid,Uruguay,1
41,Real Madrid,Germany,1
37,Real Madrid,Brazil,5
38,Real Madrid,Croatia,1
39,Real Madrid,Dominican Republic,1
40,Real Madrid,France,3
36,Real Madrid,Belgium,2
42,Real Madrid,Norway,1
43,Real Madrid,Serbia,1
44,Real Madrid,Spain,6


In [12]:
html_table = nationality_group.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Club</th>\n      <th>Nationality</th>\n      <th>Count</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Chelsea FC</td>\n      <td>Belgium</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Chelsea FC</td>\n      <td>Brazil</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Chelsea FC</td>\n      <td>Croatia</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Chelsea FC</td>\n      <td>Denmark</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Chelsea FC</td>\n      <td>England</td>\n      <td>9</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Chelsea FC</td>\n      <td>France</td>\n      <td>3</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Chelsea FC</td>\n      <td>Germany</td>\n      <td>3</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td

In [18]:
nationality_group.to_html('Data_table.html')

## Foreign Player Minutes

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Premier League
league = "premier-league"
abbr = "GB1"
r = requests.get(f"https://www.transfermarkt.co.uk/{league}/legionaereeinsaetze/wettbewerb/{abbr}/saison_id/2020/altersklasse/alle/option/spiele/plus/1", 
                 headers= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0"})
soup = BeautifulSoup(r.content, "html.parser")

In [3]:
results = soup.find("table",class_="items")
premier_leauge_minutes = pd.read_html(str(results))[0]
premier_leauge_minutes.drop(columns=['wappen',"% minutes foreign players"],inplace=True)
premier_leauge_minutes.columns=['Club', 'Players used', 'Non-foreigners played',
       'Used foreign players', '% minutes non-foreigners',
       '% minutes foreign players']
premier_leauge_minutes = premier_leauge_minutes[:20]
premier_leauge_minutes = premier_leauge_minutes.sort_values(by="% minutes foreign players", ascending=False)
premier_leauge_minutes

,Club,Players used,Non-foreigners played,Used foreign players,% minutes non-foreigners,% minutes foreign players
19,Wolves,27,4,23,13.6 %,86.4 %
18,Spurs,24,6,18,20.1 %,79.9 %
17,Man City,24,6,18,21.8 %,78.2 %
16,Arsenal,29,8,21,22.7 %,77.3 %
15,Chelsea,27,8,19,26.1 %,73.9 %
14,Liverpool,28,8,20,26.4 %,73.6 %
13,Fulham,28,7,21,29.7 %,70.3 %
12,Leicester,27,8,19,31.9 %,68.1 %
11,Leeds,23,7,16,33.5 %,66.5 %
10,Crystal Palace,24,10,14,36.5 %,63.5 %


In [12]:
engine = create_engine('sqlite:///football.db')

In [13]:
premier_leauge_minutes.to_sql('premier_league_foreign_minutes', con=engine, if_exists='replace', index=False)

In [14]:
engine.execute("SELECT * FROM premier_league_foreign_minutes").fetchall()

[('Wolves', '27', 4, 23, '13.6 %', '86.4 %'),
 ('Spurs', '24', 6, 18, '20.1 %', '79.9 %'),
 ('Man City', '24', 6, 18, '21.8 %', '78.2 %'),
 ('Arsenal', '29', 8, 21, '22.7 %', '77.3 %'),
 ('Chelsea', '27', 8, 19, '26.1 %', '73.9 %'),
 ('Liverpool', '28', 8, 20, '26.4 %', '73.6 %'),
 ('Fulham', '28', 7, 21, '29.7 %', '70.3 %'),
 ('Leicester', '27', 8, 19, '31.9 %', '68.1 %'),
 ('Leeds', '23', 7, 16, '33.5 %', '66.5 %'),
 ('Crystal Palace', '24', 10, 14, '36.5 %', '63.5 %'),
 ('West Ham', '24', 8, 16, '36.7 %', '63.3 %'),
 ('Man Utd', '29', 10, 19, '40.6 %', '59.4 %'),
 ('Everton', '29', 10, 19, '41.1 %', '58.9 %'),
 ('Newcastle', '27', 11, 16, '42.1 %', '57.9 %'),
 ('Brighton', '27', 8, 19, '43.9 %', '56.1 %'),
 ('Sheff Utd', '27', 15, 12, '49.5 %', '50.5 %'),
 ('West Brom', '30', 16, 14, '53.3 %', '46.7 %'),
 ('Southampton', '29', 13, 16, '53.9 %', '46.1 %'),
 ('Aston Villa', '24', 12, 12, '55.3 %', '44.7 %'),
 ('Burnley', '25', 15, 10, '75.1 %', '24.9 %')]

In [9]:
# Ligue 1
league = "ligue-1"
abbr = "FR1"

r = requests.get(f"https://www.transfermarkt.co.uk/{league}/legionaereeinsaetze/wettbewerb/{abbr}/saison_id/2020/altersklasse/alle/option/spiele/plus/1", 
                 headers= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0"})
soup = BeautifulSoup(r.content, "html.parser")

In [10]:
results = soup.find("table",class_="items")
ligue_one_minutes = pd.read_html(str(results))[0]
ligue_one_minutes.drop(columns=['wappen',"% minutes foreign players"],inplace=True)
ligue_one_minutes.columns=['Club', 'Players used', 'Non-foreigners played',
       'Used foreign players', '% minutes non-foreigners',
       '% minutes foreign players']
ligue_one_minutes = ligue_one_minutes[:20]
ligue_one_minutes = ligue_one_minutes.sort_values(by="% minutes foreign players", ascending=False)
ligue_one_minutes

,Club,Players used,Non-foreigners played,Used foreign players,% minutes non-foreigners,% minutes foreign players
19,Stade Reims,29,7,22,19.5 %,80.5 %
18,Paris SG,33,10,23,21.4 %,78.6 %
17,Olympique Lyon,29,10,19,22.1 %,77.9 %
16,FC Metz,27,9,18,26.0 %,74.0 %
15,Dijon,33,13,20,27.5 %,72.5 %
14,LOSC Lille,21,8,13,37.4 %,62.6 %
13,OGC Nice,31,17,14,49.2 %,50.8 %
12,Nîmes Olympique,34,18,16,49.4 %,50.6 %
11,R. Strasbourg,26,12,14,50.8 %,49.2 %
10,G. Bordeaux,28,14,14,52.2 %,47.8 %


In [15]:
ligue_one_minutes.to_sql('ligue__one_foreign_minutes', con=engine, if_exists='replace', index=False)

In [18]:
engine.execute("SELECT * FROM ligue__one_foreign_minutes").fetchall()

[('Stade Reims', '29', 7, 22, '19.5 %', '80.5 %'),
 ('Paris SG', '33', 10, 23, '21.4 %', '78.6 %'),
 ('Olympique Lyon', '29', 10, 19, '22.1 %', '77.9 %'),
 ('FC Metz', '27', 9, 18, '26.0 %', '74.0 %'),
 ('Dijon', '33', 13, 20, '27.5 %', '72.5 %'),
 ('LOSC Lille', '21', 8, 13, '37.4 %', '62.6 %'),
 ('OGC Nice', '31', 17, 14, '49.2 %', '50.8 %'),
 ('Nîmes Olympique', '34', 18, 16, '49.4 %', '50.6 %'),
 ('R. Strasbourg', '26', 12, 14, '50.8 %', '49.2 %'),
 ('G. Bordeaux', '28', 14, 14, '52.2 %', '47.8 %'),
 ('Marseille', '31', 18, 13, '52.8 %', '47.2 %'),
 ('Lens', '27', 14, 13, '57.9 %', '42.1 %'),
 ('FC Nantes', '25', 14, 11, '59.5 %', '40.5 %'),
 ('SCO Angers', '31', 19, 12, '60.4 %', '39.6 %'),
 ('Montpellier', '25', 15, 10, '60.5 %', '39.5 %'),
 ('Saint-Étienne', '40', 28, 12, '60.6 %', '39.4 %'),
 ('Monaco', '30', 13, 17, '61.5 %', '38.5 %'),
 ('FC Lorient', '27', 16, 11, '64.2 %', '35.8 %'),
 ('Stade Rennais', '32', 22, 10, '67.7 %', '32.3 %'),
 ('Stade Brest 29', '26', 19, 7, '77.

In [6]:
# La Liga
league = "laliga"
abbr = "ES1"
r = requests.get(f"https://www.transfermarkt.co.uk/{league}/legionaereeinsaetze/wettbewerb/{abbr}/saison_id/2020/altersklasse/alle/option/spiele/plus/1", 
                 headers= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0"})
soup = BeautifulSoup(r.content, "html.parser")

In [7]:
results = soup.find("table",class_="items")
laliga_minutes = pd.read_html(str(results))[0]
laliga_minutes.drop(columns=['wappen',"% minutes foreign players"],inplace=True)
laliga_minutes.columns=['Club', 'Players used', 'Non-foreigners played',
       'Used foreign players', '% minutes non-foreigners',
       '% minutes foreign players']
laliga_minutes = laliga_minutes[:20]
laliga_minutes = laliga_minutes.sort_values(by="% minutes foreign players", ascending=False)
laliga_minutes

,Club,Players used,Non-foreigners played,Used foreign players,% minutes non-foreigners,% minutes foreign players
19,Real Madrid,30,14,16,28.3 %,71.7 %
18,Atlético Madrid,25,8,17,29.7 %,70.3 %
17,Sevilla FC,26,10,16,30.4 %,69.6 %
16,Valencia,30,11,19,35.2 %,64.8 %
15,Barcelona,25,11,14,37.0 %,63.0 %
14,Getafe,31,13,18,41.0 %,59.0 %
13,Granada CF,34,20,14,48.1 %,51.9 %
12,Real Betis,25,14,11,51.7 %,48.3 %
11,SD Eibar,30,21,9,61.2 %,38.8 %
10,Cádiz CF,34,23,11,62.1 %,37.9 %


In [16]:
laliga_minutes.to_sql('laliga_foreign_minutes', con=engine, if_exists='replace', index=False)

In [19]:
engine.execute("SELECT * FROM laliga_foreign_minutes").fetchall()

[('Real Madrid', '30', 14, 16, '28.3 %', '71.7 %'),
 ('Atlético Madrid', '25', 8, 17, '29.7 %', '70.3 %'),
 ('Sevilla FC', '26', 10, 16, '30.4 %', '69.6 %'),
 ('Valencia', '30', 11, 19, '35.2 %', '64.8 %'),
 ('Barcelona', '25', 11, 14, '37.0 %', '63.0 %'),
 ('Getafe', '31', 13, 18, '41.0 %', '59.0 %'),
 ('Granada CF', '34', 20, 14, '48.1 %', '51.9 %'),
 ('Real Betis', '25', 14, 11, '51.7 %', '48.3 %'),
 ('SD Eibar', '30', 21, 9, '61.2 %', '38.8 %'),
 ('Cádiz CF', '34', 23, 11, '62.1 %', '37.9 %'),
 ('Celta de Vigo', '30', 19, 11, '66.4 %', '33.6 %'),
 ('Elche CF', '30', 18, 12, '68.2 %', '31.8 %'),
 ('Real Valladolid', '32', 24, 8, '72.2 %', '27.8 %'),
 ('Levante', '29', 20, 9, '72.2 %', '27.8 %'),
 ('Alavés', '30', 21, 9, '79.3 %', '20.7 %'),
 ('Villarreal', '29', 19, 10, '79.8 %', '20.2 %'),
 ('Real Sociedad', '30', 25, 5, '80.3 %', '19.7 %'),
 ('SD Huesca', '27', 20, 7, '82.7 %', '17.3 %'),
 ('CA Osasuna', '28', 22, 6, '84.5 %', '15.5 %'),
 ('Athletic', '27', 26, 1, '99.9 %', '0.1 %